In [1]:
import pandas as pd
from sqlite3 import connect

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Создайте соединение с базой данных с помощью библиотеки sqlite3.**

In [20]:
con = connect('/content/drive/MyDrive/Школа 21/Проверка/project11/data/checking-logs.sqlite')

**2. Создайте новую таблицу datamart в базе данных, объединив таблицы pageviews и checker с помощью только одного запроса.**

*   Таблица должна содержать следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
*   first_commit_ts — это просто новое имя для столбца timestamp из таблицы checker; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
*   first_view_ts — первое посещение пользователем из таблицы pageviews, метка времени посещения пользователем ленты новостей.
*   По-прежнему нужно использовать фильтр status = 'ready'.
*   По-прежнему нужно использовать фильтр numTrials = 1.
*   Имена лабораторных заданий по-прежнему должны быть из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1.
*   Таблица должна содержать только пользователей (uid с user_*), а не администраторов.
*   first_commit_ts и first_view_ts должны быть распарсены как datetime64[ns]

In [21]:
sql_command = '''
SELECT
    q1.uid,
    labname,
    first_commit_ts,
    first_view_ts
FROM
    (
        SELECT
            uid,
            timestamp AS first_commit_ts,
            labname
        FROM
            checker
        WHERE
            status = 'ready'
            and numTrials = 1
            and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
            and uid like 'user_%'
    ) AS q1
    LEFT Join (
        SELECT
            uid,
            min(datetime) AS first_view_ts
        FROM
            pageviews
        GROUP BY
            uid
    ) AS q2 on q1.uid = q2.uid
'''

In [22]:
datamart = pd.read_sql(sql_command, con, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035


In [23]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [24]:
datamart.to_sql('datamart', con=con, if_exists='replace') #сохранение datamart

In [25]:
pd.read_sql('PRAGMA table_info(datamart);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


**3. Используя методы библиотеки Pandas, создайте два датафрейма: test и control.**
*   test должен включать пользователей, у которых имеются значения в first_view_ts.
*   control должен включать пользователей, у которых отсутствуют значения в first_view_ts.
*   Замените пропущенные значения в control средним значением first_view_ts пользователей из test (оно пригодится нам для анализа в будущем).
*   Сохраните обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [26]:
test = datamart[pd.notnull(datamart['first_view_ts'])]
test

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [27]:
control = datamart[pd.isnull(datamart['first_view_ts'])]
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,NaT
132,user_6,laba06s,2020-05-20 14:50:07.609937,NaT
134,user_7,laba06s,2020-05-20 23:05:37.742597,NaT
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT


In [28]:
avr = test.first_view_ts.mean()
avr

Timestamp('2020-04-27 00:40:05.761783552')

In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [30]:
control['first_view_ts'] = avr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


In [32]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [33]:
test.to_sql('test', con=con, if_exists='replace')
control.to_sql('control', con=con, if_exists='replace')

**4. Закройте соединение.**

In [34]:
con.close()